In [26]:
import pandas as pd
dataSet= pd.read_csv("biodegradable_a.csv", sep=",")
dataSet

,SpMax_L,J_Dz(e),nHM,F01,F04,NssssC,nCb,C,nCp,nO,...,C_026,F02_CN,nHDon,SpMax_B,Psi_i_A,nN,SM6_B,nArCOOR,nX,Biodegradable
0,3.919000,2.690900,0.0,0.0,0.0,0.0,0.0,31.400000,2.0,0.0,...,0.0,0.0,0.0,2.949000,1.591000,0.0,7.253000,0.0,0.0,RB
1,4.170000,2.114400,0.0,0.0,0.0,0.0,0.0,30.800000,1.0,1.0,...,0.0,0.0,0.0,3.315000,1.967000,0.0,7.257000,0.0,0.0,RB
2,3.932000,3.251200,0.0,0.0,0.0,0.0,0.0,26.700000,2.0,4.0,...,0.0,0.0,1.0,NaN,2.417000,0.0,7.601000,0.0,0.0,RB
3,3.000000,2.709800,0.0,0.0,0.0,0.0,0.0,20.000000,NaN,2.0,...,0.0,0.0,1.0,NaN,5.000000,0.0,6.690000,0.0,0.0,RB
4,4.236000,3.394400,0.0,0.0,0.0,0.0,0.0,29.400000,2.0,4.0,...,0.0,0.0,0.0,3.351000,2.405000,0.0,8.003000,0.0,NaN,RB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4559,4.175650,3.454649,0.0,0.0,0.0,0.0,0.0,27.300000,2.0,2.0,...,0.0,0.0,0.0,3.398382,3.241090,0.0,7.406748,0.0,0.0,RB
4560,3.853731,2.747142,0.0,NaN,0.0,0.0,0.0,33.300000,0.0,0.0,...,0.0,0.0,1.0,3.429148,2.719309,0.0,7.511511,0.0,0.0,RB
4561,4.294771,3.471226,0.0,0.0,0.0,0.0,0.0,28.422154,2.0,2.0,...,0.0,0.0,0.0,3.351666,2.729456,0.0,7.699324,0.0,0.0,RB
4562,4.560376,3.896390,0.0,0.0,0.0,0.0,0.0,NaN,2.0,3.0,...,0.0,0.0,0.0,NaN,3.059198,0.0,7.908802,0.0,0.0,RB


### Remover os espaços em branco do DataSet:

##### KNNInputer

In [27]:
import pandas as pd
from sklearn.impute import KNNImputer

# Escolhemos 5 vizinhos de modo a obter uma valor médio para cada valor em falta 
# ou seja nem muito adaptado aos dados (overfitting) nem muito suave (underfitting)
imputer = KNNImputer(n_neighbors=5, weights="uniform");
badColumns = dataSet.columns[dataSet.isnull().any()];

print(badColumns)

dataSet[badColumns] = imputer.fit_transform(dataSet[badColumns]);

dataSet

Index(['F01', 'C', 'nCp', 'HyWi_B', 'F03_CO', 'Me', 'nCIR', 'SpMax_A', 'SdO',
       'nCrt', 'SpMax_B', 'Psi_i_A', 'nX'],
      dtype='object')


,SpMax_L,J_Dz(e),nHM,F01,F04,NssssC,nCb,C,nCp,nO,...,C_026,F02_CN,nHDon,SpMax_B,Psi_i_A,nN,SM6_B,nArCOOR,nX,Biodegradable
0,3.919000,2.690900,0.0,0.0,0.0,0.0,0.0,31.400000,2.0,0.0,...,0.0,0.0,0.0,2.949000,1.591000,0.0,7.253000,0.0,0.0,RB
1,4.170000,2.114400,0.0,0.0,0.0,0.0,0.0,30.800000,1.0,1.0,...,0.0,0.0,0.0,3.315000,1.967000,0.0,7.257000,0.0,0.0,RB
2,3.932000,3.251200,0.0,0.0,0.0,0.0,0.0,26.700000,2.0,4.0,...,0.0,0.0,1.0,3.183760,2.417000,0.0,7.601000,0.0,0.0,RB
3,3.000000,2.709800,0.0,0.0,0.0,0.0,0.0,20.000000,1.0,2.0,...,0.0,0.0,1.0,3.091871,5.000000,0.0,6.690000,0.0,0.0,RB
4,4.236000,3.394400,0.0,0.0,0.0,0.0,0.0,29.400000,2.0,4.0,...,0.0,0.0,0.0,3.351000,2.405000,0.0,8.003000,0.0,0.0,RB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4559,4.175650,3.454649,0.0,0.0,0.0,0.0,0.0,27.300000,2.0,2.0,...,0.0,0.0,0.0,3.398382,3.241090,0.0,7.406748,0.0,0.0,RB
4560,3.853731,2.747142,0.0,0.0,0.0,0.0,0.0,33.300000,0.0,0.0,...,0.0,0.0,1.0,3.429148,2.719309,0.0,7.511511,0.0,0.0,RB
4561,4.294771,3.471226,0.0,0.0,0.0,0.0,0.0,28.422154,2.0,2.0,...,0.0,0.0,0.0,3.351666,2.729456,0.0,7.699324,0.0,0.0,RB
4562,4.560376,3.896390,0.0,0.0,0.0,0.0,0.0,31.872380,2.0,3.0,...,0.0,0.0,0.0,3.509342,3.059198,0.0,7.908802,0.0,0.0,RB


Categorizar a variável Biodegradable em binário.

In [28]:
binario = pd.get_dummies(dataSet, columns=['Biodegradable'], prefix=['Biodegradable']);

dataSet = pd.concat([dataSet, binario['Biodegradable_RB']], axis=1)
dataSet = dataSet.drop('Biodegradable', axis=1)


dataSet


,SpMax_L,J_Dz(e),nHM,F01,F04,NssssC,nCb,C,nCp,nO,...,C_026,F02_CN,nHDon,SpMax_B,Psi_i_A,nN,SM6_B,nArCOOR,nX,Biodegradable_RB
0,3.919000,2.690900,0.0,0.0,0.0,0.0,0.0,31.400000,2.0,0.0,...,0.0,0.0,0.0,2.949000,1.591000,0.0,7.253000,0.0,0.0,1
1,4.170000,2.114400,0.0,0.0,0.0,0.0,0.0,30.800000,1.0,1.0,...,0.0,0.0,0.0,3.315000,1.967000,0.0,7.257000,0.0,0.0,1
2,3.932000,3.251200,0.0,0.0,0.0,0.0,0.0,26.700000,2.0,4.0,...,0.0,0.0,1.0,3.183760,2.417000,0.0,7.601000,0.0,0.0,1
3,3.000000,2.709800,0.0,0.0,0.0,0.0,0.0,20.000000,1.0,2.0,...,0.0,0.0,1.0,3.091871,5.000000,0.0,6.690000,0.0,0.0,1
4,4.236000,3.394400,0.0,0.0,0.0,0.0,0.0,29.400000,2.0,4.0,...,0.0,0.0,0.0,3.351000,2.405000,0.0,8.003000,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4559,4.175650,3.454649,0.0,0.0,0.0,0.0,0.0,27.300000,2.0,2.0,...,0.0,0.0,0.0,3.398382,3.241090,0.0,7.406748,0.0,0.0,1
4560,3.853731,2.747142,0.0,0.0,0.0,0.0,0.0,33.300000,0.0,0.0,...,0.0,0.0,1.0,3.429148,2.719309,0.0,7.511511,0.0,0.0,1
4561,4.294771,3.471226,0.0,0.0,0.0,0.0,0.0,28.422154,2.0,2.0,...,0.0,0.0,0.0,3.351666,2.729456,0.0,7.699324,0.0,0.0,1
4562,4.560376,3.896390,0.0,0.0,0.0,0.0,0.0,31.872380,2.0,3.0,...,0.0,0.0,0.0,3.509342,3.059198,0.0,7.908802,0.0,0.0,1


### Scaling data...

##### StandartScaler


In [38]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


# Dizemos então que X sao as features e o Y é a variavel de classificação
X = dataSet.drop('Biodegradable_RB', axis=1)
y = dataSet['Biodegradable_RB']

# Faz-se a divisao entre teste e treino 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

#Escalam-se os dados...
scaler = StandardScaler()
toScale = X_train.columns
scaler.fit(X_train[toScale])

# Escalam-se os dados de treino 
X_train[toScale] = scaler.transform(X_train[toScale])

# Escalam-se os dados de teste
X_test[toScale] = scaler.transform(X_test[toScale])

print(y_test)
print()
print(X_test)



221     1
429     0
531     0
3642    1
3918    1
       ..
2981    1
2186    1
2559    1
852     1
789     0
Name: Biodegradable_RB, Length: 1507, dtype: uint8

       SpMax_L   J_Dz(e)       nHM       F01       F04    NssssC       nCb  \
221   0.656952 -0.550779 -0.243600 -0.079714 -0.209222 -0.136018 -0.636896   
429   0.083878  0.193279  1.067467 -0.079714 -0.209222 -0.136018  0.807995   
531  -2.343988 -1.548641  2.378535 -0.079714 -0.209222 -0.136018 -0.636896   
3642 -0.315070  0.571371 -0.243600 -0.079714 -0.209222 -0.136018 -0.636896   
3918 -0.695097  0.430511 -0.243600 -0.079714 -0.209222 -0.136018 -0.636896   
...        ...       ...       ...       ...       ...       ...       ...   
2981 -0.320767  0.556944 -0.243600 -0.079714 -0.209222 -0.136018 -0.636896   
2186 -0.167962  1.528914 -0.243600 -0.079714 -0.209222 -0.136018 -0.636896   
2559  0.657367 -0.038544 -0.243600 -0.079714  1.389760 -0.136018  0.807995   
852   0.700087  0.476699 -0.243600 -0.079714 -0.209222 -0.